# Homework 2 Part 2: Charbel Yammine

The second portion of Homework 2.

## Preliminaries

In [1]:
# To auto-reload modules in jupyter notebook (so that changes in files *.py doesn't require manual reloading):
# https://stackoverflow.com/questions/5364050/reloading-submodules-in-ipython
%load_ext autoreload
%autoreload 2

Import commonly used libraries and magic command for inline plotting

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os

from openpyxl import Workbook, load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment
from openpyxl.utils import get_column_letter

In [3]:
%matplotlib inline

## Work

This is where work gets done.

In [4]:
import os
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
import pandas as pd

# Step 1: Combine all CSV files into a single Excel workbook
folder_path = "logs"
output_filename = "BCM.xlsx"

# Create a new workbook
workbook = Workbook()
workbook.remove(workbook.active)

# Get a list of all CSV files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith(".csv")]

# Initialize min_date and max_date variables
min_date = None
max_date = None

# Iterate over each CSV file
for csv_file in csv_files:
    # Read the CSV file into a DataFrame
    file_path = os.path.join(folder_path, csv_file)
    sheet_name = os.path.splitext(csv_file)[0]
    df = pd.read_csv(file_path, header=None, names=["datetime", "scale", "temperature"])

    # Determine min_date and max_date from the DataFrame
    current_min_date = df["datetime"].min()
    current_max_date = df["datetime"].max()

    # Update min_date and max_date if necessary
    if min_date is None or current_min_date < min_date:
        min_date = current_min_date
    if max_date is None or current_max_date > max_date:
        max_date = current_max_date

    # Create a new worksheet in the workbook
    worksheet = workbook.create_sheet(sheet_name)

    # Write the column headers to the worksheet
    headers = ["datetime", "scale", "temperature"]
    worksheet.append(headers)

    # Write the data from the DataFrame to the worksheet
    for row in dataframe_to_rows(df, index=False, header=False):
        worksheet.append(row)

# Save the workbook
workbook.save(output_filename)


# Step 2: Add formulas to each sheet
workbook = load_workbook(output_filename)  # Use load_workbook function

# Iterate over each sheet in the workbook
for sheet_name in workbook.sheetnames:
    worksheet = workbook[sheet_name]

    # Get the last row with data
    last_row = worksheet.max_row

    # Add labels for minimum, maximum, and average temperature in column G
    labels = ["Minimum", "Maximum", "Average"]
    for i, label in enumerate(labels, start=2):
        cell = worksheet.cell(row=i, column=7)
        cell.value = label
        cell.alignment = Alignment(horizontal="right")

    # Add formulas for minimum, maximum, and average temperature in column H
    column_letter = get_column_letter(8)  # Get the letter for column H
    for i, column in enumerate(["C", "C", "C"], start=2):
        formula_min = f"=MIN({column}2:{column}{last_row})"
        formula_max = f"=MAX({column}2:{column}{last_row})"
        formula_avg = f"=AVERAGE({column}2:{column}{last_row})"
        for j, formula in enumerate([formula_min, formula_max, formula_avg], start=2):
            cell = worksheet.cell(row=j, column=8)
            cell.value = formula
            cell.alignment = Alignment(horizontal="right")
            cell.number_format = '0.00'

    # Add formulas for minimum and maximum datetime in column H
    formula_min_datetime = f"=MIN(A2:A{last_row})"
    formula_max_datetime = f"=MAX(A2:A{last_row})"
    worksheet.cell(row=6, column=8).value = formula_min_datetime
    worksheet.cell(row=7, column=8).value = formula_max_datetime
    worksheet.cell(row=6, column=7).value = "Min Datetime"
    worksheet.cell(row=7, column=7).value = "Max Datetime"
    worksheet.cell(row=6, column=7).alignment = Alignment(horizontal="right")
    worksheet.cell(row=7, column=7).alignment = Alignment(horizontal="right")

    # Apply formatting to column H
    for row in worksheet[f"{column_letter}2:{column_letter}4"]:
        for cell in row:
            cell.number_format = '0.00'

    # Update the formulas for minimum and maximum datetime with the determined min_date and max_date
    worksheet.cell(row=6, column=8).value = min_date
    worksheet.cell(row=7, column=8).value = max_date

# Save the modified workbook
workbook.save(output_filename)
